## Imports

In [148]:
import numpy as np
import pandas as pd
from ape import accounts, chain, Contract, networks, project
import time

In [149]:
networks.parse_network_choice('ethereum:mainnet:alchemy').__enter__()

<alchemy chain_id=1>

## Contract objects

In [150]:
# Load Uniswap factory contract
factory = Contract("0x1F98431c8aD98523631AE4a59f267346ea31F984")

# Load a referece pool; obtain `contract_type` from here to read any uniswap pool with ape
pool_ref = Contract('0x8f8EF111B67C04Eb1641f5ff19EE54Cda062f163')
time.sleep(2)
# Required pool
pool = Contract("0x2f62f2b4c5fcd7570a709dec05d68ea19c82a9ec", contract_type=pool_ref.contract_type)

token0 = Contract(pool.token0())
token1 = Contract(pool.token1())

token0_decimals = token0.decimals()
token1_decimals = token1.decimals()

## Get prices

#### Inputs for querying and saving price data

In [151]:
start_block = 12388028  # Deployment block of required pool
# start_block = chain.blocks.head.number - 100_000
head_block = chain.blocks.head.number
step = 2400
filename = f"data/price_{pool.address}_{start_block}_{head_block}_{step}.csv"

#### Helpful functions

In [153]:
def price(sqrtPriceX96):
    y_by_x = (sqrtPriceX96**2) * (10**(token0_decimals)) // 2**(96*2)
    x_by_y = (10 ** (token0_decimals + token1_decimals)) // y_by_x
    return y_by_x, x_by_y

#### Query prices

In [154]:
# query for tick data each block and insert into pandas dataframe
is_head = True
y_decimals = token1.decimals()
for block in range(start_block, head_block, step):
    print(f"Processing block {block} ...", end='\r')
    
    # get the sqrt price data at block
    slot0 = pool.slot0(block_identifier=block)
    y_by_x, x_by_y = price(slot0.sqrtPriceX96)
    row = {
        'block_number': [block],
        'timestamp': [chain.blocks[block].timestamp],
        'sqrt_price_x96': [slot0.sqrtPriceX96],
        'y/x': [y_by_x],
        'x/y': [x_by_y]
    }
        
    # convert to pd dataframe then append to file
    df = pd.DataFrame(data=row)
    df.to_csv(filename, mode='a', index=False, header=is_head)
        
    if is_head:
        is_head = False

In [155]:
df = pd.read_csv(filename)
df.head()

,block_number,timestamp,sqrt_price_x96,y/x,x/y
0,12388028,1620402493,2108889692158436052392065,708514203,1411404310267581184960381097
1,12390428,1620433963,2275600869295620741930066,824960233,1212179642118579550973337644
2,12392828,1620466094,3573284485497124640947326,2034117424,491613703418136592295371832
3,12395228,1620497699,4912614678281149830635472,3844733444,260096054659023586655699504
4,12397628,1620529528,5528778279236812307604963,4869666057,205352890381986207724880137


In [156]:
df.shape

(2343, 5)